# Imports

In [1]:
import pandas as pd

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from collections import Counter

# Dataset Inspection

In [2]:
# Load the cleaned data
df = pd.read_csv('clean_df.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2892 entries, 0 to 2891
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   quantity         2586 non-null   float64
 1   unit             2524 non-null   object 
 2   ingredient       2890 non-null   object 
 3   ingredient_step  2892 non-null   int64  
 4   recipe_id        2892 non-null   int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 113.1+ KB


In [4]:
# Group by recipe_id and aggregate ingredients into lists
grouped_df = df.groupby('recipe_id')['ingredient'].apply(list).reset_index()

# Display grouped DataFrame
grouped_df

,recipe_id,ingredient
0,0,"[dark rum, lime juice, sugar, angostura, perno..."
1,1,"[silver tequila, marie brizard creme de cacao,..."
2,2,"[gin, mari brizard white creme de cacao, lille..."
3,3,"[pernod, sugar, water]"
4,4,"[gold tequila, gold rum, grapefruit juice, pin..."
...,...,...
650,650,"[light rum, galliano, triple sec, lime juice]"
651,651,"[dark rum, light rum, tia maria, orange juice,..."
652,652,"[vodka, galliano, lime juice, pineapple juice]"
653,653,"[yellow chartreuse, pernod, apricot brandy]"


In [5]:
# Extract the list of lists of ingredients
recipes = grouped_df['ingredient'].tolist()

# Get the list of all unique ingredients
all_ingredients = df['ingredient'].unique().tolist()

# Use MultiLabelBinarizer to encode the ingredients
mlb = MultiLabelBinarizer(classes=all_ingredients)
encoded_recipes = mlb.fit_transform(recipes)

# Create a DataFrame for easier manipulation
encoded_df = pd.DataFrame(encoded_recipes, columns=mlb.classes_)

In [6]:
encoded_df.shape

(655, 389)

# Outline:

Model Building:

Our model architecture is:

l0: 389 (amount of unique ingredients)

hidden layers:          relu

L :            sigmoid   , 389

In [7]:
input_dim = len(all_ingredients)  # Number of unique ingredients

In [8]:


model = Sequential()
model.add(Dense(units=128, input_dim=input_dim, activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=input_dim, activation='sigmoid'))  # Output layer with sigmoid activation

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
# attempt at at a 2nd model, larger architecture since 389 param

input_dim = len(all_ingredients)  # Number of unique ingredients

model = Sequential()
model.add(Dense(units=256, input_dim=input_dim, activation='relu'))  # First hidden layer
model.add(Dense(units=128, activation='relu'))  # Second hidden layer
model.add(Dense(units=64, activation='relu'))  # Second hidden layer
model.add(Dense(units=128, activation='relu'))  # Second hidden layer
model.add(Dense(units=256, activation='relu'))  # Third hidden layer
model.add(Dense(units=input_dim, activation='sigmoid'))  # Output layer

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
# Print model summary to verify the architecture
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 256)               99840     
                                                                 
 dense_4 (Dense)             (None, 128)               32896     
                                                                 
 dense_5 (Dense)             (None, 64)                8256      
                                                                 
 dense_6 (Dense)             (None, 128)               8320      
                                                                 
 dense_7 (Dense)             (None, 256)               33024     
                                                                 
 dense_8 (Dense)             (None, 389)               99973     
                                                                 
Total params: 282,309
Trainable params: 282,309
Non-tr

In [11]:
# Attemping dropout layers
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Define the model
model = Sequential()
model.add(Dropout(0.1, input_shape=(input_dim,)))  # Dropout layer for the input layer
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))  # Dropout layer for hidden layers
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))  # Dropout layer for hidden layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Dropout layer for hidden layers
model.add(Dense(input_dim, activation='sigmoid'))  # Output layer for multi-label classification


# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout (Dropout)           (None, 389)               0         
                                                                 
 dense_9 (Dense)             (None, 512)               199680    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_10 (Dense)            (None, 256)               131328    
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_11 (Dense)            (None, 128)               32896     
                                                                 
 dropout_3 (Dropout)         (None, 128)              

Model Training:

In [12]:
X = encoded_df.values  # Input features (binary vectors of ingredients)
y = encoded_df.values  # Target labels (same as input for multi-label classification)

#model.fit(X, y, epochs=20, batch_size=50, validation_split=0.2)

Generating New Recipes:

Start with a seed ingredient or set of ingredients.
Use the trained model to predict additional ingredients.

In [13]:
model.fit(X, y, epochs=50, batch_size=50, validation_split=0.2)

seed_ingredients = ['vodka']  # Example seed ingredients
seed_vector = mlb.transform([seed_ingredients])[0]

predicted_probabilities = model.predict(seed_vector.reshape(1, -1))
predicted_ingredients = mlb.inverse_transform((predicted_probabilities > 0.00001).astype(int))[0]

generated_recipe = list(set(seed_ingredients + list(predicted_ingredients)))
print("Generated Recipe:", generated_recipe)


Epoch 1/50
11/11 [==============================] - 1s 25ms/step - loss: 0.6438 - accuracy: 0.0019 - val_loss: 0.5030 - val_accuracy: 0.0000e+00
Epoch 2/50
11/11 [==============================] - 0s 7ms/step - loss: 0.2883 - accuracy: 0.0076 - val_loss: 0.0719 - val_accuracy: 0.0000e+00
Epoch 3/50
11/11 [==============================] - 0s 10ms/step - loss: 0.0918 - accuracy: 0.0802 - val_loss: 0.0838 - val_accuracy: 0.1832
Epoch 4/50
11/11 [==============================] - 0s 13ms/step - loss: 0.0812 - accuracy: 0.1183 - val_loss: 0.0589 - val_accuracy: 0.1908
Epoch 5/50
11/11 [==============================] - 0s 14ms/step - loss: 0.0648 - accuracy: 0.0668 - val_loss: 0.0520 - val_accuracy: 0.2290
Epoch 6/50
11/11 [==============================] - 0s 18ms/step - loss: 0.0598 - accuracy: 0.0725 - val_loss: 0.0517 - val_accuracy: 0.2519
Epoch 7/50
11/11 [==============================] - 0s 12ms/step - loss: 0.0581 - accuracy: 0.1126 - val_loss: 0.0522 - val_accuracy: 0.1832
Epoch 

In [14]:
# Number of times to generate recipes
num_iterations = 100

# Example seed ingredients
seed_ingredients = ['gin', 'lime juice']
seed_vector = mlb.transform([seed_ingredients])[0]

# List to store generated recipes
generated_recipes = []

for _ in range(num_iterations):
    # Fit the model for one epoch
    model.fit(X, y, epochs=5, batch_size=389, validation_split=0.2, verbose=0)
    
    # Predict probabilities
    predicted_probabilities = model.predict(seed_vector.reshape(1, -1))
    
    # Convert probabilities to ingredients
    predicted_ingredients = mlb.inverse_transform((predicted_probabilities > 0.001).astype(int))[0]
    
    # Generate recipe
    generated_recipe = list(set(seed_ingredients + list(predicted_ingredients)))
    
    # Store the generated recipe as a tuple for easier counting
    generated_recipes.append(tuple(generated_recipe))

# Count the occurrences of each unique recipe
recipe_counts = Counter(generated_recipes)

# Convert to DataFrame
df = pd.DataFrame(recipe_counts.items(), columns=['Recipe', 'Count'])

# Sort by count (optional)
df = df.sort_values(by='Count', ascending=False)

print(df)


1/1 [==============================] - 0s 29ms/step
                                               Recipe  Count
0   (green chartreuse, absinth, orange flower wate...      1
63  (green chartreuse, galliano, peach bitters, wa...      1
73  (green chartreuse, galliano, luxardo maraschin...      1
72  (green chartreuse, galliano, luxardo maraschin...      1
71  (green chartreuse, galliano, luxardo maraschin...      1
..                                                ...    ...
30  (green chartreuse, orange flower water, waterm...      1
29  (green chartreuse, absinth, orange flower wate...      1
28  (green chartreuse, orange flower water, waterm...      1
27  (green chartreuse, absinth, orange flower wate...      1
99  (green chartreuse, luxardo maraschino, limonce...      1

[100 rows x 2 columns]


In [15]:
df['Recipe']

0     (green chartreuse, absinth, orange flower wate...
63    (green chartreuse, galliano, peach bitters, wa...
73    (green chartreuse, galliano, luxardo maraschin...
72    (green chartreuse, galliano, luxardo maraschin...
71    (green chartreuse, galliano, luxardo maraschin...
                            ...                        
30    (green chartreuse, orange flower water, waterm...
29    (green chartreuse, absinth, orange flower wate...
28    (green chartreuse, orange flower water, waterm...
27    (green chartreuse, absinth, orange flower wate...
99    (green chartreuse, luxardo maraschino, limonce...
Name: Recipe, Length: 100, dtype: object

In [16]:
#df['Recipe'][0]

In [17]:
import numpy as np

# Assuming `predicted_probabilities` is already defined and is a 2D array
# and `mlb` is your MultiLabelBinarizer instance

# Get the indices that would sort each row of the array in descending order
sorted_indices = np.argsort(-predicted_probabilities, axis=1)

# Get the sorted probabilities
sorted_probabilities = np.take_along_axis(predicted_probabilities, sorted_indices, axis=1)

# For each row, map the sorted indices back to the ingredient names
# Assuming you want to do this for the first row in the predictions
top_ingredients_indices = sorted_indices[0]
top_probabilities = sorted_probabilities[0]

# Map these indices back to the ingredient names
top_ingredients = mlb.classes_[top_ingredients_indices]

# Print the top ingredients and their corresponding probabilities
for ingredient, probability in zip(top_ingredients, top_probabilities):
    print(f"Ingredient: {ingredient}, Probability: {probability}")


Ingredient: gin, Probability: 0.9839162826538086
Ingredient: lime juice, Probability: 0.9655337929725647
Ingredient: luxardo maraschino, Probability: 0.03960534185171127
Ingredient: green chatreuse, Probability: 0.039571747183799744
Ingredient: cointreau, Probability: 0.027259111404418945
Ingredient: guava nectar, Probability: 0.019158106297254562
Ingredient: apricot brandy, Probability: 0.015707489103078842
Ingredient: orange curaçao, Probability: 0.015610558912158012
Ingredient: simple, Probability: 0.014942003414034843
Ingredient: simple syrup, Probability: 0.013629039749503136
Ingredient: lemon juice, Probability: 0.013035033829510212
Ingredient: grenadine, Probability: 0.01165839470922947
Ingredient: light rum, Probability: 0.007835473865270615
Ingredient: club soda, Probability: 0.006077099125832319
Ingredient: orange juice, Probability: 0.005745273549109697
Ingredient: sugar, Probability: 0.005074561573565006
Ingredient: grapefruit juice, Probability: 0.0043823448941111565
Ingre

Summary of Steps:
Clean Data Preparation:

Create a DataFrame with binary encoded ingredient vectors for each recipe.
Model Building:

Define a neural network to learn from these vectors.
Model Training:

Train the model on your dataset.
Recipe Generation:

Generate new recipes starting from seed ingredients.
By following these steps, you should be able to create and train a model that can generate new cocktail recipes based on the patterns it learns from your dataset. Let me know if you need more detail on any of these steps!

The architecture described here is a simple feedforward neural network, also known as a Multi-Layer Perceptron (MLP). This architecture is used for multi-label classification, where each ingredient is treated as a binary label that can be present or absent in a recipe.

Architecture Explanation
Input Layer:

The input layer consists of nodes equal to the number of unique ingredients. Each node represents whether a particular ingredient is present (1) or absent (0) in the recipe.
Input Dimension: input_dim = len(all_ingredients).
Hidden Layers:

The network has two hidden layers, each with 128 neurons. These layers use the ReLU (Rectified Linear Unit) activation function, which introduces non-linearity to the model and allows it to learn complex patterns.
First Hidden Layer: Dense(units=128, input_dim=input_dim, activation='relu')
Second Hidden Layer: Dense(units=128, activation='relu')
Output Layer:

The output layer also consists of nodes equal to the number of unique ingredients. Each node represents the probability of the corresponding ingredient being part of the recipe.
The sigmoid activation function is used in the output layer to produce probabilities between 0 and 1 for each ingredient.
Output Layer: Dense(units=input_dim, activation='sigmoid')
Loss Function:

The model uses binary cross-entropy loss, suitable for multi-label classification where each label (ingredient) is a binary decision (present or absent).
Loss Function: binary_crossentropy
Optimizer:

The Adam optimizer is used to minimize the loss function during training. Adam is a popular choice due to its adaptive learning rate and efficiency.
Optimizer: adam
Metrics:

The accuracy metric is used to evaluate the model's performance during training and validation.
Metrics: accuracy
Why This Architecture?
Simplicity: This architecture is straightforward and easy to implement, making it suitable for a dataset of your size (654 recipes and 447 unique ingredients).
Flexibility: The use of dense layers allows the network to learn from the presence and absence of each ingredient, capturing the relationships between different ingredients.
Scalability: Adding more hidden layers or increasing the number of neurons in each layer can improve the model's capacity to learn more complex patterns, if needed.
Summary
This architecture is a feedforward neural network designed for multi-label classification. It takes a binary vector representing the presence or absence of each ingredient as input and outputs a binary vector representing the predicted probabilities of each ingredient being part of the cocktail recipe. This approach is suitable for generating new recipes by learning the common ingredient combinations from the training data.








## Next chunk of code: Randomly decides to include this ingredient based on its probability

In [18]:
import numpy as np
from collections import Counter
import pandas as pd

# Assuming `predicted_probabilities` is already defined and is a 2D array
# and `mlb` is your MultiLabelBinarizer instance

def generate_semi_random_cocktail(predicted_probabilities, mlb, top_n=10, threshold=0.05):
    # Ensure predicted_probabilities is 2D
    if predicted_probabilities.ndim == 1:
        predicted_probabilities = predicted_probabilities.reshape(1, -1)
    
    # Get the indices that would sort each row of the array in descending order
    sorted_indices = np.argsort(-predicted_probabilities, axis=1)
    
    # Get the sorted probabilities
    sorted_probabilities = np.take_along_axis(predicted_probabilities, sorted_indices, axis=1)

    # For each row, map the sorted indices back to the ingredient names
    # Assuming you want to do this for the first row in the predictions
    top_ingredients_indices = sorted_indices[0]
    top_probabilities = sorted_probabilities[0]

    # Map these indices back to the ingredient names
    top_ingredients = mlb.classes_[top_ingredients_indices]

    # Semi-random selection based on sorted probabilities
    selected_ingredients = []
    for i in range(top_n):
        if top_probabilities[i] > threshold:
            # Randomly decide to include this ingredient based on its probability
            if (np.random.random() * 0.1) < top_probabilities[i]:
                selected_ingredients.append(top_ingredients[i])
    
    return selected_ingredients

# Number of times to generate recipes
num_iterations = 100

# Example seed ingredients
seed_ingredients = ['gin', 'lime juice']
seed_vector = mlb.transform([seed_ingredients])[0]

# List to store generated recipes
generated_recipes = []

for _ in range(num_iterations):
    # Fit the model for one epoch
    model.fit(X, y, epochs=1, batch_size=389, validation_split=0.2, verbose=0)
    
    # Predict probabilities
    predicted_probabilities = model.predict(seed_vector.reshape(1, -1))
    
    # Generate recipe using the semi-random selection method
    generated_recipe = generate_semi_random_cocktail(predicted_probabilities, mlb, top_n=10, threshold=0.000000005)
    
    # Add the seed ingredients to ensure they are included
    full_recipe = list(set(seed_ingredients + generated_recipe))
    
    # Store the generated recipe as a tuple for easier counting
    generated_recipes.append(tuple(full_recipe))

# Count the occurrences of each unique recipe
recipe_counts = Counter(generated_recipes)

# Convert to DataFrame
df = pd.DataFrame(recipe_counts.items(), columns=['Recipe', 'Count'])

# Sort by count (optional)
df = df.sort_values(by='Count', ascending=False)

print(df)


1/1 [==============================] - 0s 24ms/step
                                               Recipe  Count
8                  (gin, green chatreuse, lime juice)     13
5               (gin, luxardo maraschino, lime juice)     11
0                                   (gin, lime juice)     10
28                       (gin, cointreau, lime juice)      7
9   (gin, green chatreuse, luxardo maraschino, lim...      5
24               (gin, simple, cointreau, lime juice)      3
10                          (gin, simple, lime juice)      3
33  (gin, lemon juice, luxardo maraschino, lime ju...      3
4       (gin, cointreau, green chatreuse, lime juice)      2
29      (gin, green chatreuse, cointreau, lime juice)      2
26  (luxardo maraschino, simple, lime juice, green...      2
23   (gin, cointreau, luxardo maraschino, lime juice)      2
1                   (gin, orange curaçao, lime juice)      2
16  (gin, simple syrup, luxardo maraschino, lime j...      2
2       (gin, simple, luxardo mar

In [19]:
df

,Recipe,Count
8,"(gin, green chatreuse, lime juice)",13
5,"(gin, luxardo maraschino, lime juice)",11
0,"(gin, lime juice)",10
28,"(gin, cointreau, lime juice)",7
9,"(gin, green chatreuse, luxardo maraschino, lim...",5
24,"(gin, simple, cointreau, lime juice)",3
10,"(gin, simple, lime juice)",3
33,"(gin, lemon juice, luxardo maraschino, lime ju...",3
4,"(gin, cointreau, green chatreuse, lime juice)",2
29,"(gin, green chatreuse, cointreau, lime juice)",2


In [20]:
all_ingredients[1]

'lime juice'

In [21]:
print("Predicted Probabilities:", predicted_probabilities)

Predicted Probabilities: [[1.36834351e-04 9.78660047e-01 4.61095525e-03 2.66485498e-03
  1.32118203e-05 6.10285788e-05 1.46152859e-04 2.08288679e-06
  1.20878872e-02 9.81802583e-01 2.29768135e-04 8.75995902e-04
  1.38480782e-05 2.60845147e-04 5.77305385e-04 4.06666379e-03
  5.48402895e-04 3.11657932e-04 8.56987572e-06 2.46367654e-05
  1.56528022e-05 2.14515580e-07 1.27729857e-02 1.26105090e-06
  3.56975498e-08 2.63844122e-04 4.48604394e-03 1.05153966e-04
  6.67288143e-04 1.84832385e-03 5.71838696e-04 2.36182197e-04
  1.25964259e-04 2.17643799e-04 9.31952382e-05 3.20381980e-04
  1.55504665e-03 1.54575810e-05 3.66133085e-04 1.28525324e-04
  4.97987820e-03 4.18063173e-05 3.30667448e-04 6.23919583e-10
  7.12917093e-03 1.15260236e-05 3.16805817e-04 4.00620251e-04
  1.02524431e-02 2.03697849e-03 5.65062328e-05 6.57242992e-07
  7.02100806e-04 8.03109560e-06 6.67178610e-05 1.99352344e-03
  8.72645105e-05 1.50746801e-05 1.68226811e-03 8.90570107e-07
  2.05842284e-07 1.70750991e-06 4.05757710e-0

In [22]:
predicted_probabilities[86]

IndexError: index 86 is out of bounds for axis 0 with size 1